### Importación de librerías

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Carga del dataset
df = pd.read_csv(r"winequality-red.csv", sep=";")

In [9]:
#Verificación del dataset
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Analisis exploratorio de los datos

In [10]:
#Visualización del shape
print(f"Shape de los datos: {df.shape}")

Shape de los datos: (1599, 12)


In [12]:
#Visualización de los tipos de datos del df
print("Tipos de datos:")
df.dtypes

Tipos de datos:


fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [13]:
#Separación del dataset
x = df.drop("quality", axis=1)
y = df["quality"]

In [14]:
#Obtención del set de entrenamiento y prueba
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=00000)

### Generación del modelo y entrenamiento

In [21]:
from sklearn.ensemble import RandomForestClassifier

#Creación del modelo
rf = RandomForestClassifier(n_estimators=3, random_state=00000)

In [22]:
#Entrenamiento
rf.fit(x_train, y_train)

#Obtención de las predicciones
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)

### Evalucación del modelo

In [23]:
from sklearn.metrics import classification_report

#Reporte con datos de entrenamiento
print("Reporte de clasificación de entrenamiento:")
print(classification_report(y_train, y_train_pred))

#Reporte con datos de testeo
print("Reporte de clasificación de testeo:")
print(classification_report(y_test, y_test_pred))

Reporte de clasificación de entrenamiento:
              precision    recall  f1-score   support

           3       0.86      0.86      0.86         7
           4       0.86      0.79      0.82        38
           5       0.90      0.96      0.93       473
           6       0.95      0.90      0.93       434
           7       0.93      0.92      0.93       154
           8       1.00      0.77      0.87        13

    accuracy                           0.92      1119
   macro avg       0.92      0.87      0.89      1119
weighted avg       0.92      0.92      0.92      1119

Reporte de clasificación de testeo:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        15
           5       0.64      0.74      0.69       208
           6       0.61      0.50      0.55       204
           7       0.37      0.47      0.41        45
           8       0.00      0.00      0.00         5